### Generating Animated Map with `folium`

https://nbviewer.jupyter.org/github/python-visualization/folium/blob/master/examples/Plugins.ipynb#Timestamped-GeoJSON

In [ ]:
import pandas as pd
import folium
from folium import plugins
import requests

pd.options.display.max_columns = None

In [ ]:
folium.Map?

In [ ]:
# df = pd.read_csv('https://earthquake.usgs.gov/earthquakes/'
#                  'feed/v1.0/summary/2.5_day.csv')
df = pd.read_csv('https://earthquake.usgs.gov/earthquakes/'
                 'feed/v1.0/summary/2.5_month.csv')

# df['time'] = pd.to_datetime(df['time'], utc=True)

df.head()

In [ ]:
# get faults
r = requests.get('https://raw.githubusercontent.com/'
                 'fraxen/tectonicplates/master/GeoJSON/'
                 'PB2002_boundaries.json')

fault_features = r.json()['features']

# for feature in fault_features:
#     feature['properties']['color'] = 'red'

In [ ]:
features = [
    {
        'type': 'Feature',
        'geometry': {
            'type': 'Point',
            'coordinates': [r['longitude'], r['latitude']],
        },
        'properties': {
            'time': r['time'][0:-1],
            'popup': (
                f"<strong>Time:</strong> {r['time']}<br>"
                f"<strong>Place:</strong> {r['place']}<br>"
                f"<strong>Magnitude:</strong> {r['mag']} {r['magType']}<br>"
                f"<strong>Depth:</strong> {r['depth']}<br>"
            ),
            'icon': 'circle',
            'iconstyle': {
                'fillOpacity': 0.5,
                'stroke': 0,
                'radius': r['mag'] * 2.5
            },
        }
    } for i, r in df.iterrows()
]

m = folium.Map(
#     location=()
    tiles='CartoDBpositron',
#     zoom_start=1,
#     no_wrap=True,
    min_zoom=1.5,
    max_zoom=5,
)

# add faults
folium.GeoJson(
    {
        'type': 'FeatureCollection',
        'features': fault_features,
    },
    style_function = lambda x: {
        'color': 'red',
        'weight': 0.5,
    }
).add_to(m)

plugins.TimestampedGeoJson(
    {
        'type': 'FeatureCollection',
        'features': features
    },
    period='PT6H', # six hour
    time_slider_drag_update=True,
    duration='PT12H',
    date_options='YYYY-MM-DD HH UTC'
).add_to(m)

folium.plugins.Fullscreen(
    position='topright',
    force_separate_button=True,
).add_to(m)

m.save('earthquakes.html')
m

In [ ]:
# from IPython.display import IFrame
# IFrame('earthquakes_2.5_day.html', width='100%', height=500)

### Pulling Data from USGS

https://earthquake.usgs.gov/fdsnws/event/1/

In [ ]:
import requests
import pandas as pd
from io import StringIO

pd.options.display.max_columns = None

In [ ]:
payload = {
    'format': 'csv', 
#     'starttime': None,  # default last 30 days
#     'endtime': '2019-06-03',  # default now
    'minmagnitude': 0,  # default null
    'limit': None,  # default null, returns 404 over 20,000
}
url = 'https://earthquake.usgs.gov/fdsnws/event/1/query'
r = requests.get(url, params=payload)

In [ ]:
r.url

In [ ]:
df = pd.read_csv(StringIO(r.text))
print(df.shape)
df.head()

if len(df) == 20000:
    # get most recent eathquake in DF
    # make a new request starting at last date
    df.append()

### Damage Data
from https://www.ngdc.noaa.gov/nndc/struts/form?t=101650&s=1&d=1

> The Significant Earthquake Database contains information on destructive earthquakes from 2150 B.C. to the present that meet at least one of the following criteria: 
> * Moderate damage (approximately $1 million or more)
> * 10 or more deaths
> * Magnitude 7.5 or greater
> * Modified Mercalli Intensity X or greater
> * the earthquake generated a tsunami

In [ ]:
dmg = pd.read_csv('https://www.ngdc.noaa.gov/nndc/struts/results?'
                  'type_0=Exact&query_0=$ID&t=101650&s=13&d=189&dfn=signif.txt',
                  sep='\t')

In [ ]:
dmg.describe()